# Prediction of thermal fractions from MIR Spectra

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import root_mean_squared_error
from copy import deepcopy

from src.data_loader import DataLoader
from src.modeling_tools import *

In [2]:
output_folder = 'output'

## Load data

In [3]:
data_loader = DataLoader()

### Load RMQS data

In [4]:
sample_data, wavelenths_MIR, mir_spectra = data_loader.run('rmqs')

Loaded 1889 spectra.
1877 spectra after removing organic soils.
1876 spectra after removing samples with RE yield < 0.7 or > 1.3.
1876 spectra after removing samples with layer_upper_limit > 30 cm.


### Load BZE data

In [5]:
sample_data_bze, _, mir_spectra_bze = data_loader.run('bze')

Loaded 463 spectra.
463 spectra after removing organic soils.
449 spectra after removing samples with RE yield < 0.7 or > 1.3.
412 spectra after removing samples with layer_upper_limit > 30 cm.


In [6]:
sample_data_bze[['layer_upper_limit', 'layer_lower_limit']].value_counts()

layer_upper_limit  layer_lower_limit
0                  10                   410
5                  10                     2
Name: count, dtype: int64

### Load Finish data

In [7]:
sample_data_finland, _, mir_spectra_finland = data_loader.run('finland')

Loaded 87 spectra.
85 spectra after removing organic soils.
81 spectra after removing samples with RE yield < 0.7 or > 1.3.
81 spectra after removing samples with layer_upper_limit > 30 cm.


### Load New Zealand data

In [8]:
sample_data_new_zealand, _, mir_spectra_new_zealand = data_loader.run('new_zealand', filter_re_yield=False)

Loaded 282 spectra.
276 spectra after removing organic soils.
276 spectra after removing samples with layer_upper_limit > 30 cm.


## Descriptive statistics of the datasets

In [17]:
from scipy.stats import skew

def get_stats(x):
    return {
        'N': len(x),
        'Min': x.min(),
        'Mean': x.mean(),
        'Median': x.quantile(0.5),
        'Max': x.max(),
        'STD': x.std(),
        'Skew': skew(x.values)
    }

In [25]:
stats_rmqs = get_stats(sample_data.stablecarbon_frac)
stats_bze = get_stats(sample_data_bze.stablecarbon_frac)
stats_finland = get_stats(sample_data_finland.stablecarbon_frac)

In [26]:
stats_summary = pd.DataFrame([stats_rmqs, stats_bze, stats_finland], index=['RMQS', 'Germany', 'FInland']).round(2)

In [27]:
stats_summary

,N,Min,Mean,Median,Max,STD,Skew
RMQS,1876,0.25,0.45,0.45,0.84,0.12,0.48
Germany,412,0.25,0.42,0.42,0.79,0.12,0.40
FInland,81,0.26,0.39,0.36,0.60,0.09,0.48


# Model training on RMQS data

## Grid search hyperparameters

In [8]:
mir_gridsearch_results = {}

In [ ]:
name = 'stablecarbon_frac_beta_reg'

parameters_dict = {
    'window_length': [5, 11, 16, 21],
    'polyorder': [2, 3],
    'deriv': [0, 1, 2],
    'n_components': [5, 10, 15, 20, 25, 30],
}

gridsearch = GridSearch()
results, best_parameters = gridsearch.run(
    name,
    parameters_dict,
    mir_spectra,
    sample_data,
    clear_display=True,
    output_folder=os.path.join(output_folder, 'grid_search', name),
)

mir_gridsearch_results[name] = gridsearch

In [9]:
best_parameters

{'n_components': 15.0, 'polyorder': 3.0, 'deriv': 1.0, 'window_length': 5.0}

In [10]:
with open(os.path.join(output_folder, 'grid_search', 'results.p'), "wb") as f:
        pickle.dump(mir_gridsearch_results, f)

In [13]:
with open(os.path.join(output_folder, 'grid_search', 'results.p'), "wb") as f:
        pickle.dump(mir_gridsearch_results, f)

<!-- ## Predictions -->

In [ ]:
best_params_stablecarbon = mir_gridsearch_results['stablecarbon_frac_beta_reg'].best_parameters

mir_default_params = {
    'shift_position': None,
    'min_wavelength': 440,    # checked visually where the signal became less noisy
    'max_wavelength': 10000,
    'window_length': int(best_params_stablecarbon['window_length']),
    'polyorder': int(best_params_stablecarbon['polyorder']),
    'deriv': int(best_params_stablecarbon['deriv']),
    'resampling_step': 2,
}
mir_n_components = {
    key: int(mir_gridsearch_results[key].best_parameters['n_components'])
    for key in mir_gridsearch_results
}

In [ ]:
mir_predictions = run_preprocessing_training_prediction(
    mir_spectra,
    mir_default_params,
    mir_n_components,
    sample_data,
    output_folder=os.path.join(output_folder, 'prediction'),
    names=list(mir_gridsearch_results.keys()),    # here it only includes stablecarbon_frac_beta_reg only, but could include more models 
    n_splits=5,
    n_repeats=10,
)

with open(os.path.join(output_folder, 'prediction', 'predictions.p'), "wb") as f:
    pickle.dump(mir_predictions, f)

## Apply to prediction of BZE

In [17]:
best_params_stablecarbon = mir_gridsearch_results['stablecarbon_frac_beta_reg'].best_parameters

mir_default_params = {
    'shift_position': None,
    'min_wavelength': 440,    # checked visually where the signal became less noisy
    'max_wavelength': 10000,
    'window_length': int(best_params_stablecarbon['window_length']),
    'polyorder': int(best_params_stablecarbon['polyorder']),
    'deriv': int(best_params_stablecarbon['deriv']),
    'resampling_step': 2,
}
mir_n_components = {
    key: int(mir_gridsearch_results[key].best_parameters['n_components'])
    for key in mir_gridsearch_results
}

In [12]:
mir_output_folder_bze = os.path.join(output_folder, 'apply_to_bze')
os.makedirs(mir_output_folder_bze, exist_ok=True)

In [ ]:
mir_predictions_bze = run_preprocessing_prediction(
    mir_spectra_bze,
    mir_default_params,
    mir_n_components,
    sample_data_bze,
    mir_predictions,
    output_folder=mir_output_folder_bze,
    spectra_original=mir_spectra,
    names=list(mir_gridsearch_results.keys()),
)


In [14]:
with open(os.path.join(mir_output_folder_bze, 'predictions.p'), "wb") as f:
        pickle.dump(mir_predictions_bze, f)

## Apply to prediction of Finland

In [9]:
mir_output_folder_finland = os.path.join(output_folder, 'apply_to_finland')
os.makedirs(mir_output_folder_finland, exist_ok=True)

In [ ]:
mir_predictions_finland = run_preprocessing_prediction(
    mir_spectra_finland,
    mir_default_params,
    mir_n_components,
    sample_data_finland,
    mir_predictions,
    output_folder=mir_output_folder_finland,
    spectra_original=mir_spectra,
    names=list(mir_predictions.keys())
)


In [22]:
with open(os.path.join(mir_output_folder_finland, 'predictions.p'), "wb") as f:
        pickle.dump(mir_predictions_finland, f)

## CORAL transfer

### BZE

In [16]:
with open(os.path.join(output_folder, 'grid_search', 'results.p'), "rb") as f:
    mir_gridsearch_results = pickle.load(f)
mir_gridsearch_results

best_params_stablecarbon = mir_gridsearch_results['stablecarbon_frac_beta_reg'].best_parameters

mir_default_params = {
    'shift_position': None,
    'min_wavelength': 440,    # checked visually where the signal became less noisy
    'max_wavelength': 10000,
    'window_length': int(best_params_stablecarbon['window_length']),
    'polyorder': int(best_params_stablecarbon['polyorder']),
    'deriv': int(best_params_stablecarbon['deriv']),
    'resampling_step': 2,
}
mir_n_components = {
    key: int(mir_gridsearch_results[key].best_parameters['n_components'])
    for key in mir_gridsearch_results
}

In [17]:
mir_output_folder_bze = os.path.join(output_folder, 'apply_to_bze_coral')
os.makedirs(mir_output_folder_bze, exist_ok=True)

In [ ]:
mir_predictions_rmqs_aligned_bze, mir_predictions_bze = run_preprocessing_coral_training_prediction(
    mir_spectra,
    mir_spectra_bze,
    mir_default_params,
    mir_n_components,
    sample_data,
    sample_data_bze,
    output_folder=mir_output_folder_bze,
    names=list(mir_gridsearch_results.keys()),
)

In [19]:
with open(os.path.join(mir_output_folder_bze, 'predictions_rmqs_aligned_bze.p'), "wb") as f:
        pickle.dump(mir_predictions_rmqs_aligned_bze, f)

with open(os.path.join(mir_output_folder_bze, 'predictions.p'), "wb") as f:
        pickle.dump(mir_predictions_bze, f)

### Finland

In [11]:
mir_output_folder_finland = os.path.join(output_folder, 'apply_to_finland_coral')
os.makedirs(mir_output_folder_finland, exist_ok=True)

In [ ]:
mir_predictions_rmqs_aligned_finland, mir_predictions_finland = run_preprocessing_coral_training_prediction(
    mir_spectra,
    mir_spectra_finland,
    mir_default_params,
    mir_n_components,
    sample_data,
    sample_data_finland,
    output_folder=mir_output_folder_finland,
    names=list(mir_gridsearch_results.keys()),
)

In [25]:
with open(os.path.join(mir_output_folder_finland, 'predictions_rmqs_aligned_finland.p'), "wb") as f:
        pickle.dump(mir_predictions_rmqs_aligned_finland, f)

with open(os.path.join(mir_output_folder_finland, 'predictions.p'), "wb") as f:
        pickle.dump(mir_predictions_finland, f)

## Compare signals before and after alignment

In [26]:
with open(os.path.join(output_folder, 'prediction', 'predictions.p'), "rb") as f:
        mir_predictions_rmqs = pickle.load(f)

with open(os.path.join(output_folder, 'apply_to_bze', 'predictions.p'), "rb") as f:
        mir_predictions_bze = pickle.load(f)

with open(os.path.join(output_folder, 'apply_to_finland', 'predictions.p'), "rb") as f:
        mir_predictions_finland = pickle.load(f)

with open(os.path.join(output_folder, 'apply_to_bze_coral', 'predictions_rmqs_aligned_bze.p'), "rb") as f:
        mir_predictions_rmqs_aligned_bze = pickle.load(f)

with open(os.path.join(output_folder, 'apply_to_finland_coral', 'predictions_rmqs_aligned_finland.p'), "rb") as f:
        mir_predictions_rmqs_aligned_finland = pickle.load(f)

In [27]:
X_rmqs = mir_predictions_rmqs['stablecarbon_frac_beta_reg'].X
X_bze = mir_predictions_bze['stablecarbon_frac_beta_reg'].X
X_finland = mir_predictions_finland['stablecarbon_frac_beta_reg'].X
X_rmqs_aligned_bze = mir_predictions_rmqs_aligned_bze['stablecarbon_frac_beta_reg'].X
X_rmqs_aligned_finland = mir_predictions_rmqs_aligned_finland['stablecarbon_frac_beta_reg'].X

In [28]:
wavelengths = mir_predictions_rmqs['stablecarbon_frac_beta_reg'].wavelengths

In [ ]:

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

alpha=0.5

axs[0].plot(wavelengths, X_bze.mean(axis=0), c='blue', label='BZE', alpha=alpha)
axs[0].plot(wavelengths, X_finland.mean(axis=0), c='red', label='Finland', alpha=alpha)
axs[1].plot(wavelengths, X_rmqs_aligned_bze.mean(axis=0), c='blue', label='RMQS aligned with BZE', alpha=alpha)
axs[1].plot(wavelengths, X_rmqs_aligned_finland.mean(axis=0), c='red', label='RMQS aligned with Finland', alpha=alpha)


for ax in axs:
    ax.set(
        title="Mean pre-processed spectra",
        xlabel="Wavelength [nm]",
    )
    ax.plot(wavelengths, X_rmqs.mean(axis=0), c='k', label='RMQS', alpha=alpha)
    ax.legend()
    # ax.xaxis.set_ticklabels(wavelengths)


plt.show()

In [30]:
from sklearn.decomposition import PCA

In [31]:
pca = PCA(n_components=2)
pca.fit(np.concatenate([X_rmqs, X_bze, X_finland]))
print(pca.explained_variance_ratio_)

[0.28044161 0.24077415]


In [32]:
X_rmqs_reduced = pca.transform(X_rmqs)
X_bze_reduced = pca.transform(X_bze)
X_finland_reduced = pca.transform(X_finland)
X_rmqs_aligned_bze_reduced = pca.transform(X_rmqs_aligned_bze)
X_rmqs_aligned_finland_reduced = pca.transform(X_rmqs_aligned_finland)

In [ ]:

fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

s = 10
alpha=0.7

axs[0].scatter(X_rmqs_reduced[:, 0], X_rmqs_reduced[:, 1], c='gray', label='RMQS', s=s, alpha=alpha)
axs[1].scatter(X_rmqs_reduced[:, 0], X_rmqs_reduced[:, 1], c='gray', label='RMQS', s=s, alpha=alpha)
axs[0].scatter(X_bze_reduced[:, 0], X_bze_reduced[:, 1], c='blue', label='BZE', s=s, alpha=alpha)
axs[0].scatter(X_finland_reduced[:, 0], X_finland_reduced[:, 1], c='red', label='Finland', s=s, alpha=alpha)
axs[1].scatter(X_rmqs_aligned_bze_reduced[:, 0], X_rmqs_aligned_bze_reduced[:, 1], c='blue', label='RMQS aligned with BZE', s=s, alpha=alpha)
axs[1].scatter(X_rmqs_aligned_finland_reduced[:, 0], X_rmqs_aligned_finland_reduced[:, 1], c='red', label='RMQS aligned with Finland', s=s, alpha=alpha)

for ax in axs:
    ax.set(
        xlabel="1st Eigenvector",
        ylabel="2nd Eigenvector",
    )

    ax.legend()

plt.show()

## Perturbation analysis

In [ ]:
with open(os.path.join(output_folder, 'prediction', 'predictions.p'), "rb") as f:
    mir_predictions_rmqs = pickle.load(f)

In [ ]:
prediction = mir_predictions_rmqs['stablecarbon_frac_beta_reg']
X_train = prediction.X_train
X_test = prediction.X_test
y_train = prediction.y_train
y_test = prediction.y_test

N = 100   # number of perturbation analyses runs

In [ ]:
rmse = []
i = 0
while i < N:
    y_train_perturbed = [np.random.normal(loc=y, scale=0.08, size=1) for y in y_train]
    y_test_perturbed = [np.random.normal(loc=y, scale=0.08, size=1) for y in y_test]
    # keep only if [0, 1] limits are respected, otherswise generate new series
    if (
        min(y_train_perturbed) < 0 or
        min(y_test_perturbed) < 0 or
        max(y_train_perturbed) > 1 or
        max(y_test_perturbed) > 1
    ):
        continue
    i += 1
    pred = deepcopy(prediction)
    pred.fit(X_train, y_train_perturbed)
    y_test_pred, _, _ = pred.predict(X_test)
    print(i, root_mean_squared_error(y_test, y_test_perturbed))
    rmse.append(root_mean_squared_error(y_test, y_test_perturbed))

In [ ]:
np.mean(rmse)

In [ ]:
np.std(rmse)